# <center> MRI Segmentation

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# импортируем требуемые бибилиотеки

import torch
from source.network import NNSegmentation
from source.helpme import show_history, show_image, calculate_pad, load_mri
import numpy as np
import torchvision
import os
from torchvision import transforms
from sklearn.model_selection import train_test_split
import glob

In [ ]:
X, y = load_mri('./data/norm_hydro/train/', 
                size=(128, 128))
y = (y == 1).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.333, shuffle=True, random_state=17)

In [ ]:
X_train.shape

## U-net

In [ ]:
from torchsummary import summary
import matplotlib.pyplot as plt

In [ ]:
# определим архитектуру сети

unet = torch.nn.Sequential(torch.nn.Conv2d(in_channels=1, 
                                               out_channels=16, 
                                               kernel_size=3, 
                                               stride=2, 
                                               padding=1),
                               torch.nn.LeakyReLU(),
                               torch.nn.BatchNorm2d(16),
                               # 64x64

                               torch.nn.Conv2d(16, 32, 3, stride=2, padding=1),
                               torch.nn.LeakyReLU(),
                               torch.nn.BatchNorm2d(32),
                               # 32x32
                               
                               
                               torch.nn.MaxPool2d(kernel_size=3, 
                                                  stride=2, 
                                                  padding=1),
                               # 16x16

                               torch.nn.Conv2d(32, 64, 3, stride=2, padding=1),
                               torch.nn.LeakyReLU(),
                               torch.nn.BatchNorm2d(64),
                               # 8x8

                               torch.nn.Conv2d(64, 128, 3, stride=2, padding=1),
                               torch.nn.LeakyReLU(),
                               torch.nn.BatchNorm2d(128),
                               # 4x4

                               # далее обратно увеличиваем spatial size
                               torch.nn.ConvTranspose2d(128, 128, 4, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(128),
                               # 8x8
                           
                               torch.nn.ConvTranspose2d(128, 128, 4, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(128),
                               # 16x16
                           
                               torch.nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(64),
                               # 32x32
                           
                               torch.nn.ConvTranspose2d(64, 64, 4, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(64),
                               # 64x64
                           
                               torch.nn.ConvTranspose2d(64, 2, 4, stride=2, padding=1),
                               # 128x128
                          )

In [ ]:
# summary(unet, (1, 128, 128))

In [ ]:
# определим модель с удобным функционалом

model = NNSegmentation(unet, lr=1e-3)

In [ ]:
model.fit(X_train, y_train, epochs=150, batch_size=256, valid_data=[X_test, y_test])

In [ ]:
model.show_history()

In [ ]:
# посмотрим на ошибку модели на тестовой части (чем меньше, тем лучше)
model.loss(X_test, y_test)

In [ ]:
model.show_predict_grid(X_test, y_test, size=3, threshold=0.3)